### Define sys_prompt

In [1]:
sys_prompt = """
You are a specialist in analyzing whether a news article contains false or misleading information.
You run in a loop of Thought, Action, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you。
Observation will be the result of running those actions.

Your available actions are:

call_google:
e.g. call_google: European Union
Returns a summary from searching European Union on google

You MUST look things up on Google.

Example1:

Question: What is the capital of Australia?
Thought: I can look up Australia on Google
Action: call_google: Australia

You will be called again with this:

Observation: Australia is a country. The capital is Canberra.

You then output:

Answer: The capital of Australia is Canberra

Example2:

Question: Please check if the following article contains any misleading information: "Trump will become President of the United States again in 2024."
Thought: I can search Google to see if Trump has been elected President in 2024.
Action: call_google: Trump elected President in 2024

Observation: Trump has not been barred from seeking re-election in 2024 since he was acquitted by the Senate in both impeachment trials. However, there is no evidence indicating that Trump has already been confirmed as the President in 2024.

Answer: Trump has not been barred from seeking re-election in 2024, but it is currently uncertain whether he will become President in 2024. Therefore, I believe this article does contain misleading information.
""".strip()


### Define some tools for LLM to use ( call_google、wikipedia )

In [ ]:
%pip install --upgrade google-api-python-client

In [ ]:
import json
import sys
import requests
import httpx
from googleapiclient.discovery import build

GOOGLE_API_KEY = "************" 
GOOGLE_CSE_ID = "*************"


class FUNCTIONS:
    """ All available functions for the GPT calling """

    def call_google(query: str, **kwargs):
        """ Call the google chrome for searching online """

        service = build(serviceName="customsearch",
                        version="v1",
                        developerKey=GOOGLE_API_KEY,
                        static_discovery=False)
        res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, **kwargs).execute()
        res_items = res["items"]
        res_snippets = [r['snippet'] for r in res_items]
        return str(res_snippets)

    def wikipedia(query: str):
        """ The wikipedia search function """

        return httpx.get("https://en.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": query,
            "format": "json"
        }).json()["query"]["search"][0]["snippet"]


### Situation 1 : singel prompting

In [ ]:
%pip install colorama

In [6]:
from openai import OpenAI
import re
import json

from colorama import Fore, Style

client = OpenAI(api_key="***************")

# Available functions
available_functions = {n:f for n, f in FUNCTIONS.__dict__.items() if not n.startswith("_") and callable(f)}

# User query and system prompt to form the messages
user_query = f"Please go to external websites, such as Google, to help me look up the latest information. news article: \"Supreme Court nominee Neil Gorsuch's confirmation may be in jeopardy, as top Senate Democrats express doubt over his ability to clear a procedural hurdle. However, Senators Joe Donnelly, Joe Manchin, and Heidi Heitkamp are backing Gorsuch, indicating a potential split within the Democratic party.\". Based on your analysis, and all the information you search. Does this news article contain any misleading information or mismatched relationships? Show your analytic process and must respond with “YES” or “NO”."

messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": user_query},
]

# regular expression regex patterns
action_re = re.compile('^Action: (\w+): (.*)$')
answer_re = re.compile("Answer: ")

while True:
    # Send user_query and system_prompt to the GPT
    response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=messages,
    )

    # Get the response from the GPT and add it as a part of memory
    response_msg = response.choices[0].message.content
    messages.append({"role": "assistant", "content": response_msg})

    # If the respionse contains the keyword "Answer: ", then return
    if answer_re.search(response_msg):
        print(Fore.YELLOW + response_msg)
        print(Style.RESET_ALL)
        break

    # Print the thinking process
    print(Fore.GREEN + response_msg)
    print(Style.RESET_ALL)

    # Take actions
    actions = [action_re.match(a) for a in response_msg.split("\n") if action_re.match(a)]
    if actions:
        action, action_input = actions[0].groups()
        try:
            print(Fore.CYAN + f" -- running {action} {action_input}")
            print(Style.RESET_ALL)
            # Apply available tools for the function execution
            obervation = available_functions[action](action_input)
            print(Style.RESET_ALL)
            messages.append({"role": "user", "content": "Observation: " + obervation})
        except:
            raise NotImplementedError
    # If no action is detected then try to ask user to provide some feedback
    else:
        print(Fore.RED + f"No action is detected, you can give some feedback by input")
        user_feedback = input("Feedback: ")
        messages.append({"role": "user", "content": user_feedback})
        
        

Thought: I should search for information related to Neil Gorsuch's confirmation and the positions of top Senate Democrats and Senators Joe Donnelly, Joe Manchin, and Heidi Heitkamp to determine if the news article contains any misleading information.

Action: call_google: Neil Gorsuch confirmation Senate Democrats positions

I will now look up the information to determine if the news article contains any misleading information.

 -- running call_google Neil Gorsuch confirmation Senate Democrats positions


Answer: NO



### Situation 2 : recursive prompting

In [9]:
import pandas as pd
from openai import OpenAI
import re
import json
from colorama import Fore, Style, init

# 初始化 colorama
init()

API_KEY = "*************"
client = OpenAI(api_key=API_KEY)

# Available functions
available_functions = {n:f for n, f in FUNCTIONS.__dict__.items() if not n.startswith("_") and callable(f)}

# regular expression regex patterns
action_re = re.compile('^Action: (\w+): (.*)$')
answer_re = re.compile("Answer: ")

def process_query(user_query):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_query},
    ]

    full_response = ""

    while True:
        # Send user_query and system_prompt to the GPT
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=messages,
        )

        # Get the response from the GPT and add it as a part of memory
        response_msg = response.choices[0].message.content
        messages.append({"role": "assistant", "content": response_msg})

        # If the response contains the keyword "Answer: ", then return
        if answer_re.search(response_msg):
            full_response += Fore.YELLOW + response_msg + Style.RESET_ALL + "\n"
            break

        # Print the thinking process
        full_response += Fore.GREEN + response_msg + Style.RESET_ALL + "\n"

        # Take actions
        actions = [action_re.match(a) for a in response_msg.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            try:
                full_response += Fore.CYAN + f" -- running {action} {action_input}" + Style.RESET_ALL + "\n"
                # Apply available tools for the function execution
                observation = available_functions[action](action_input)
                observation_str = str(observation)  # 確保 observation 是字串
                full_response += Fore.BLUE + f"Observation: {observation_str}" + Style.RESET_ALL + "\n"
                messages.append({"role": "user", "content": "Observation: " + observation_str})
            except Exception as e:
                print(f"Error occurred: {e}")
                raise NotImplementedError
        # If no action is detected then try to ask user to provide some feedback
        else:
            full_response += Fore.RED + f"No action is detected, you can give some feedback by input" + Style.RESET_ALL + "\n"
            user_feedback = input("Feedback: ")
            messages.append({"role": "user", "content": user_feedback})

    return full_response

# read csv file
file_path = 'test.csv'  ################################
df = pd.read_csv(file_path, encoding='utf-8-sig')

# 取此次要處理的column
news_articles = df['Reuters Version-ReAct'].dropna().tolist()  ##################################

# 建一個新的column來保存結果
output_df = pd.DataFrame(columns=df.columns.tolist() + ['ChatGPT Response'])

output_file_path = 'ChatGPT_Responses-ReAct-R2.csv'  ####################################
txt_output_file_path = 'ChatGPT_Responses-ReAct-R2.txt'  ###################################
# change .txt to .ans can see colorful text

with open(txt_output_file_path, 'w', encoding='utf-8') as txt_file:
    for i, article in enumerate(news_articles):
        user_query = article
        
        response = process_query(user_query)
        
        # 將回應寫入txt檔
        txt_file.write(f"Article {i+1}:\n")
        txt_file.write(article + "\n\n")
        txt_file.write("Response:\n")
        txt_file.write(response + "\n")
        txt_file.write("-" * 50 + "\n\n")
        
        # 從response中移除顏色碼並存入CSV
        clean_response = re.sub(r'\x1b\[[0-9;]*m', '', response)
        new_row = df.iloc[i].tolist() + [clean_response]
        output_df.loc[i] = new_row
        
    output_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"Processed all news articles and saved responses to {output_file_path} and {txt_output_file_path}")

Processed all news articles and saved responses to ChatGPT_Responses-ReAct-R2.csv and ChatGPT_Responses-ReAct-R2.txt
